<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ocp_load_pretrained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import os
if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .

Cloning into 'ocp'...
remote: Enumerating objects: 8687, done.
remote: Counting objects: 100% (912/912), done.
remote: Compressing objects: 100% (437/437), done.
remote: Total 8687 (delta 585), reused 730 (delta 459), pack-reused 7775
Receiving objects: 100% (8687/8687), 101.43 MiB | 19.17 MiB/s, done.
Resolving deltas: 100% (5850/5850), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ocp-models (pyproject.toml) ... done
  Created wheel for ocp-models: filename=ocp_models-0.0.3-0.editable-py3-none-any.whl size=3294 sha256=2425a7a6c9ea6de6693522a4ab405ebb82ca961f5f1e1d9eb51172c1cb1521a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-jzupzdw8/wheels/29/db/87/61b8ac706262b

In [5]:
pip install  lmdb pymatgen==2023.5.10 ase==3.22.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os, torch
from ase.io import read
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import SchNet

ERROR:root:Invalid setup for SCN. Either the e3nn library or Jd.pt is missing.


In [7]:
!wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/is2re/schnet_all.pt

--2023-06-02 01:39:23--  https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/is2re/schnet_all.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.38, 13.35.7.128, 13.35.7.82, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14643781 (14M) [binary/octet-stream]
Saving to: ‘schnet_all.pt’

schnet_all.pt       100%[===================>]  13.96M  14.1MB/s    in 1.0s    

2023-06-02 01:39:24 (14.1 MB/s) - ‘schnet_all.pt’ saved [14643781/14643781]



In [8]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
attrs = torch.load("schnet_all.pt", map_location=device)["config"][
    "model_attributes"
]

In [9]:
model = SchNet(
    None,
    None,
    1,
    cutoff=6.0,
    hidden_channels=attrs["hidden_channels"],
    num_interactions=attrs["num_interactions"],
    num_gaussians=attrs["num_gaussians"],
    regress_forces=False,
    use_pbc=True,
)

state_dict = torch.load("schnet_all.pt", map_location=device)["state_dict"]
state_dict = {k[2 * len("module.") :]: v for k, v in state_dict.items()}
model.load_state_dict(state_dict, strict=True)
model.to(device)

SchNetWrap(hidden_channels=384, num_filters=128, num_interactions=4, num_gaussians=100, cutoff=6.0)

In [10]:
import os
if not os.path.exists('jarvis_leaderboard'):
  !git clone https://github.com/usnistgov/jarvis_leaderboard.git
os.chdir('jarvis_leaderboard')
!pip install -e .

Cloning into 'jarvis_leaderboard'...
remote: Enumerating objects: 35441, done.
remote: Counting objects: 100% (4241/4241), done.
remote: Compressing objects: 100% (586/586), done.
remote: Total 35441 (delta 2148), reused 3979 (delta 2014), pack-reused 31200
Receiving objects: 100% (35441/35441), 280.03 MiB | 25.47 MiB/s, done.
Resolving deltas: 100% (18316/18316), done.
Updating files: 100% (2324/2324), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ocp/jarvis_leaderboard
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from jarvis.core.atoms import Atoms
from jarvis.core.specie import atomic_numbers_to_symbols
import os, torch
from ase.io import read
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from jarvis.core.atoms import Atoms
from tqdm import tqdm
from jarvis.db.figshare import  get_request_data
import json,zipfile
import numpy as np
import pandas as pd
from jarvis.db.jsonutils import loadjson,dumpjson
import os
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import Atoms
from jarvis.core.specie import atomic_numbers_to_symbols
from ocpmodels.datasets import SinglePointLmdbDataset
import os, torch
from ase.io import read
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import CGCNN,SchNet
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from jarvis.core.atoms import Atoms
from ocpmodels.datasets import data_list_collater

In [ ]:
dat = get_request_data(js_tag="ocp_all.json",url="https://figshare.com/ndownloader/files/40974599")
df = pd.DataFrame(dat)
# from https://github.com/usnistgov/jarvis_leaderboard/tree/main/jarvis_leaderboard/benchmarks/AI/SinglePropertyPrediction/ocp10k_relaxed_energy.json.zip
path = "../../../../benchmarks/AI/SinglePropertyPrediction/ocp_all_relaxed_energy.json.zip"
js_tag = "ocp_all_relaxed_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()))
val_ids = np.array(list(id_data['val'].keys()))
test_ids = np.array(list(id_data['test'].keys()))
train_df = df[df['id'].isin(train_ids)]
val_df = (df[df['id'].isin(val_ids)])#[:take_val]
test_df = (df[df['id'].isin(test_ids)])

100%|██████████| 657M/657M [01:30<00:00, 7.26MiB/s]


Loading the zipfile...


In [ ]:
model.eval()

In [ ]:
f=open('AI-SinglePropertyPrediction-relaxed_energy-all-test-mae.csv','w')
f.write('id,target,prediction\n')

for ii,i in tqdm(test_df.iterrows()):
    fname=i['id']
    atoms=(Atoms.from_dict(i['atoms'])).ase_converter()
    actual=i['relaxed_energy']
    relaxed_energy = (actual-target_mean)/target_std
    scaled=relaxed_energy
    data = a2g.convert(atoms).to(device)
    batch = data_list_collater([data], otf_graph=False)
    out = model(batch)
    pred=(out[0].detach().cpu().numpy().flatten().tolist()[0])*target_std+target_mean
    line=str(fname)+','+str(actual)+','+str(pred) #+'\n'
    #line=str(i.sid)+','+str(actual)+','+str(scaled)+','+str(pred) #+'\n'
    f.write(line+'\n')
f.close()

In [ ]:
model = SchNet(
    None,
    None,
    1,
    cutoff=6.0,
    hidden_channels=attrs["hidden_channels"],
    num_interactions=attrs["num_interactions"],
    num_gaussians=attrs["num_gaussians"],
    regress_forces=False,
    use_pbc=True,
)

model.load_state_dict(
    torch.load("schnet_all.pt", map_location=device)["state_dict"]
)
model.to(device)

RuntimeError: ignored